# Abaco4DSP AWG system example

## Table of contents
* [Imports setup and initialisation](#setup)
  * [Connecting to the system prior to initialization](#connect)
  * [Initialization](#init)
  * [Trigger setup](#trigger_setup) 
* [Uploading a pulse sequence](#upload)
  * [Via the sequencer and template element](#sequencer)
  * [Uploading a forged sequence](#fs)
* [More complex triggering sequences](#burst_triggers)  

## Imports <a class="anchor" id="setup"></a>

In [ ]:
from qcodes.instrument_drivers.Abaco import Abaco4DSP

# for the parametric sequencer, if using it to generate, forge and upload sequence
from qdev_wrappers.customised_instruments.parametric_sequencer import ParametricSequencer
from qdev_wrappers.customised_instruments.AWGinterface import Abaco4DSPInterface
from pulse_building.readout_template_element import create_readout_template_element

In [ ]:
# for parametric sequencer, if using

import yaml
import sys

scriptfolder = 'A:\Scripts'
sys.path.append(scriptfolder)
with open(scriptfolder + 'pulse_building/pulse_building_defaults.yaml') as f:
    initial_sequence_settings = yaml.safe_load(f)

## Connecting to the Abaco4DSP <a class="anchor" id="connect"></a>
In order to connect to the instrument, you will need to do a couple of extra things, beyond just knowing its IP address and port (the port is theoretically fixed, but if ever it seems not to connect, follow "finding the port" steps below). Initializing the instrument will fail if these things have not been done first.

You need to open the program that allows it to run on the Abaco control computer. Additionally, if you have not done so in the past, you will need to connect to the waveform file folder with the measurement computer, so that you can send waveform files to the instrument. 

### Opening the program on the Abaco4DSP
You will need to either attach a monitor and keyboard, or connect remotely to the Abaco control computer. You will need to do this any time the Abaco control computer restarts. Once you are logged on to the control computer (username: PCX79470, password: W10w00rd!), you will need to:

1. Open the file PRJ0161_WorkstationApp in Microsoft Visual Studio.
2. Click the green "Play" triangle that says something about running the system in debug mode.
3. A GUI will open. You need to select the box marked 'Use Remote Control'. The 4DSP system will now begin listening for and responding to commands on its port. 

N.B. Do not trust the box that appears to let you enter a port number. This box does NOT allow you to set the port the Abaco system uses. It is purely decorative. If you need the port number, see the section further down on finding the port. 

### Connecting to the waveform file folder from the measurement computer

If this is the first time you are connecting with this measurement computer, you will need to allow your measurement computer to access the folder where the waveform files are saved on the Abaco4DSP control computer. Otherwise, you can skip this step.

The file is set as shared on the Abaco4DSP computer. Details about this?

You should see the Abaco4DSP control computer under *Network* when you open File Explorer. Then what did we do???????

### Connection problems: Finding the port

If the IP address is correct, and the system is not connecting, check that you are broadcasting on the correct port:

1. Open Task Manager on the Abaco4DSP control computer
2. Go to the Performance tab
3. Click the option at the bottom to *Open Resource Monitor*. 
4. In the Resource Monitor, go to the Network tab
5. Go to *Listening Ports*, and find the entry for *PRJ0161_WorkstationApp.exe*. This should give you the port number.

## Initialization <a class="anchor" id="setup"></a>

In [ ]:
#initialize - initialize interface, Abaco and ps

abaco = Abaco4DSP('Abaco4DSP', 192.168.20.117, port=27015)

# for parametric sequencer
awg_interface = Abaco4DSPInterface(abaco)
initial_template_element = create_readout_template_element()
ps = ParametricSequencer('parametric_sequencer',
                            awg=awg_interface,
                            template_element=initial_template_element,
                            inner_setpoints='dummy_param', [1],
                            context=initial_sequence_settings['context'],
                            labels=initial_sequence_settings['labels'],
                            units=initial_sequence_settings['units'])

## Basic trigger setup  <a class="anchor" id="trigger_setup"></a>
    
You will need a trigger input for the Abaco4DSP. For each trigger, a single element of the sequence is output.

In this example, the Agilent33522A AWG was used to provide the triggers. The settings on the trigger source were adjusted manually. Output was set to pulses of 1.5V, with 1 us duration. 

*N.B. Because trigger spacing has to be longer than the total pulse duration (to allow time for upload to the FPGA), and every pulse requires a trigger, it is not possible for the Abaco4DSP to output continuously. There will always be deadtime between sequence elements.*

#### Trigger frequency

The system will output only one element per trigger. The distance from the beginning of one pulse to the beginning of the next is set by trigger frequency. When uploading to the Abaco4DSP using the sequencer, the *total_duration* will set the element length, but not the spacing between pulses.

If the trigger frequency is too high, the triggers will not provide  sufficient time for the Abaco system to upload the next pulse to the FPGA before recieving the trigger, and the system will not output at all. The theoretical maximum trigger frequency (based on system specifications) is determined by the the length of the longest element. We will look at this once the sequence is uploaded. For 20us pulses, the maximum theoretical frequency is 38 kHz. Setting it a few kHz below the theoretical maximum is advisable.

#### Monitoring output

It is recommended to reserve a test channel outputting a known signal going to the oscilloscope, and to connect the trigger output (labeled *TRIG PRI.* on the panel for channels 1-16) to the oscilloscope as well. In this way, you can monitor whether the Abaco is successfully outputting an undistorted sequence element for each trigger. Since there are no analogue channels to monitor, this is the closest you can come to confirming that the system is outputting correctly with out beginning to dismantle your setup in order to measure the channels you are actively using.

## Uploading a pulse sequence <a class="anchor" id="upload"></a>

The Abaco 4DSP driver currently lacks a function taking only waveform amplitudes as a list - the upload function takes a forged lomentum sequence, of the format:

In [ ]:
[{'data' : {1: array1,
            2: array2, 
            3: array3, ...},
  'sequencing' : {'nrep': 1}}]

The Abaco4DSP can use the 'nrep' information from the lomentum sequence by accomodating it in the uploaded file, but it doesn't support other sequencing information, such as jump_to, goto_state or trig_wait. If this information is contained in the forged sequence, it will be ignored.

*Note on peak-to-peak amplitude: The default peak-to-peak amplitude of the Abaco4DSP system is 1.7 V. This is also the maximum output of the system. The amplitude specified in the forged sequence is therefore relative to 1.7 V - so an amplitude of 0.8 does not output 0.8 V, but 1.36 V.*



### Using the sequencer <a class="anchor" id="sequencer"></a>

First we make a template element - in this case sidebanding two mixers with two different frequencies, for the entirity of the drive_stage_duration.

We also include a 'test' channel, connected to the oscilloscope to monitor whether the system is outputting correctly.

In [ ]:
from lomentum import Element, SegmentGroup
from lomentum.atoms import zero, sine

def create_2_mixer_template_element():
    
    seg1 = zero(duration='drive_stage_duration')
    seg2 = sine(duration='readout_duration', 
                frequency=50000, 
                amplitude=1, 
                phase=0, 
                offset=0)
    seg3 = zero(duration='after_readout_duration')
    test = SegmentGroup(seg1, seg2, seg3, duration='total_duration') 
    
    seg1 = sine(duration='drive_stage_duration', 
                frequency='mixer1_frequency', 
                amplitude='mixer1_amplitude_I', 
                phase=0, 
                offset='mixer1_offset_I')
    seg2 = zero(duration='readout_duration')
    seg3 = zero(duration='after_readout_duration')
    Mixer1_I = SegmentGroup(seg1, seg2, seg3, duration='total_duration')    

    seg1 = sine(duration='drive_stage_duration', 
                frequency='mixer1_frequency', 
                amplitude='mixer1_amplitude', 
                phase='mixer1_phase', 
                offset = 'mixer1_offset_Q')
    seg2 = zero(duration='readout_duration')
    seg3 = zero(duration='after_readout_duration')
    Mixer1_Q = SegmentGroup(seg1, seg2, seg3, duration='total_duration')
    
    seg1 = sine(duration='drive_stage_duration', 
                frequency='mixer2_frequency', 
                amplitude='mixer2_amplitude_I', 
                phase=0, 
                offset='mixer2_offset_I')
    seg2 = zero(duration='readout_duration')
    seg3 = zero(duration='after_readout_duration')
    Mixer2_I = SegmentGroup(seg1, seg2, seg3, duration='total_duration')    

    seg1 = sine(duration='drive_stage_duration', 
                frequency='mixer2_frequency', 
                amplitude='mixer2_amplitude', 
                phase='mixer2_phase', 
                offset = 'mixer2_offset_Q')
    seg2 = zero(duration='readout_duration')
    seg3 = zero(duration='after_readout_duration')
    Mixer2_Q = SegmentGroup(seg1, seg2, seg3, duration='total_duration')
  
    def mytransformation(context):
        context['after_readout_duration'] = context['total_duration'] - \
            context['drive_stage_duration'] - context['readout_duration']
        context['mixer1_amplitude_I'] = context['mixer1_amplitude']*context['mixer1_relative_amplitude_I']
        context['mixer2_amplitude_I'] = context['mixer2_amplitude']*context['mixer2_relative_amplitude_I']
    
    
    template_element = Element(segments={1: test, 
                                         2: Mixer1_I, 
                                         3: Mixer1_Q,
                                         4: Mixer2_I,
                                         5: Mixer2_Q,
                               sequencing={'nrep': 1},
                               transformation=mytransformation)
    
    return template_element

We then provide the template element and the needed context for this pulse upload to the sequencer:

In [ ]:
mixer_context = {
 'drive_readout_delay': 0,
 'drive_stage_duration': 10e-6, 
 'dummy_param': 1,
 'marker_duration': 5e-08,
 'mixer1_amplitude': 0.162,
 'mixer2_amplitude': 0.189,
 'mixer1_relative_amplitude_I': 1,
 'mixer2_relative_amplitude_I': 1,
 'mixer1_frequency' : 75e6,
 'mixer2_frequency' : 100e6,
 'mixer1_offset_I': 0,
 'mixer1_offset_Q': 0,
 'mixer2_offset_I': 0,
 'mixer2_offset_Q': 0,
 'mixer1_phase': -np.pi/2,
 'mixer2_phase': -np.pi/2,
 'readout_duration': 4e-6,
 'total_duration':20e-6}

template_element = create_2_mixer_template_element()

ps.set_template(template_element,
            inner_setpoints=('dummy_time', [1]),
            context=mixer_context)

Once this is done, each time the system recieves a trigger, it will output the next element in the sequence. This will fail if it recieves triggers with too high a frequency. In this case, it is not enough to reduce trigger frequency to the correct range - you will need to reupload the sequence to reset the system.

### Uploading a forged sequence directly <a class="anchor" id="fs"></a>

If you are not using the sequencer, but are creating a forged sequence in some other way, this can be given directly to the upload function. I'm not going to do that, so lets just take the sequence object the sequencer just made for us, and treat it as a forged sequence we have created and want to upload:

In [ ]:
forged_sequence = ps._sequence_object.forge(SR=1e9, context=ps._sequence_context)

# convert the forged sequence into a file of the correct format and save
abaco.make_and_send_awg_file(forged_sequence)

# load the saved waveform file
abaco.load_waveform_from_file(abaco.FILENAME)

This is the point (before telling it to run) that you should be sure that you aren't sending triggers at too high a frequency. If the system starts receiving triggers at too high a frequency, the output system will stop functioning, and will need to be reset by running *load_waveform_from_file* again.

If you are in doubt about the correct trigger frequency for your currently uploaded sequence, you can ask it:

In [ ]:
abaco.max_trigger_freq()

This is the **theoretical** maximum trigger frequency, based on the system memory specifications. In practice, the cutoff seems to often be a few kHz below the maximum. Lets set our trigger frequency in this case to 35kHz. Then:

In [ ]:
# start outputting
abaco.run()

Unless asked to do otherwise, the function *make_and_send_awg_file* uses the default file name, and overwrites whatever file was most recently uploaded with the default file name. If you wanted to save the sequence to use again later, you could instead run:

In [ ]:
abaco.make_and_send_awg_file(forged_sequence, 'my_file')

And then to use the file later:

In [ ]:
abaco.load_waveform_from_file('my_file')
abaco.run()

## Limitations of the Agilent3325 - outputting in burst mode <a class="anchor" id="burst_triggers"></a>

If you wish to output in bursts, for example in order to allow MIDAS time to flush the buffer every 2048 pulses, the Agilent33522A is not a suitable trigger source, because it continues to output with high impedance when outputting in "burst" mode, which confuses the 4DSP and results in unwanted pulses being output. This makes the pulse uploads a bit more complicated, because the Tektronix also needs a pulse sequence uploaded. See "Example: Abaco4DSP with Tektronix5014C triggers" for an example of doing this with the Tektronix5014C to provide the trigger bursts.